In [118]:
import praw # Python Reddit API Wrapper
#Praw documentation: https://praw.readthedocs.io/en/stable/index.html
import pandas as pd


# Create a Reddit instance
# This is a read-only instance since we only need to read data
# We don't need it to be authenticated since we aren't posting anything
reddit = praw.Reddit(
    client_id='51f3IBDpHGRzaGh0FGj0ng',
    client_secret='C-mvx1K5Flj_4kM-Grc9FLT2_bHxag',
    user_agent='TradingAnalysisApp by /u/Ok_Ability_9624',
)



In [119]:
print(reddit.read_only)

True


In [120]:
subreddit = reddit.subreddit("wallstreetbets")
print(subreddit.display_name)  
print(subreddit.title)
print(subreddit.description)

wallstreetbets
wallstreetbets
The rules and submission guidelines are maintained on new Reddit so be sure to check them and make sure you're up to date.

* Read the [rules](https://www.reddit.com/r/wallstreetbets/about/rules)

* Read the [comment and submission guide](https://www.reddit.com/r/wallstreetbets/wiki/contentguide)

* Read the [FAQ](https://www.reddit.com/r/wallstreetbets/wiki/faq) if you're new to both wallstreetbets and trading.

---
**Join the discord**

[WSB Discord](https://discord.gg/wsbverse)

**filter by flairs**

^Navigate ^WSB |^We ^recommend ^best ^daily ^DD
:--|:--     
**DD** | [All](https://ns.reddit.com/r/wallstreetbets/search?sort=new&restrict_sr=on&q=flair%3ADD) / [**Best Daily**](https://ns.reddit.com/r/wallstreetbets/search?sort=top&q=flair%3ADD&restrict_sr=on&t=day) / [Best Weekly](https://ns.reddit.com/r/wallstreetbets/search?sort=top&q=flair%3ADD&restrict_sr=on&t=week)
**Discussion** | [All](https://ns.reddit.com/r/wallstreetbets/search?sort=new&restric

In [121]:

stockCSV = pd.read_csv('stock_info.csv') #Read the CSV file

stockCSV['Ticker'] = '$' + stockCSV['Ticker'] #Add a $ sign in front of the ticker
stockCSV.head() #Display the first 5 rows of the dataframe


,Ticker,Name,Exchange
0,$A,Agilent Technologies,NYSE
1,$AA,Alcoa Inc.,NYSE
2,$AAN,Aaron's Inc,NYSE
3,$AAT,American Assets Trust,NYSE
4,$AAV,Advantage Oil & Gas Ltd,NYSE


In [ ]:

postData = []
import nltk  #Natural Language Toolkit
nltk.download('punkt') #sentence word tokenziation
import textstat #a simple python library to calculate statistics from text 
from textblob import TextBlob
from nltk.tokenize import TabTokenizer
from nltk.tokenize import word_tokenize
import datetime
buySell = 0
 #word tokenizer

# Fetch the top 1000 hot posts in the 'wallstreetbets' subreddit
for submission in subreddit.hot(limit=1000):
        #textWords = nltk.word_tokenize(submission.selftext) #tokenize the post text, pretty sure we do not need this
        wordsPost = submission.selftext.split() #split the title into words
        blob = TextBlob(submission.selftext) #create a textblob object
        
        #determining buy sell depending on sentiment
        if(blob.sentiment.polarity >0):
            buySell = 1
        elif(blob.sentiment.polarity <0):
            buySell = -1
        else:
            buySell = 0
        
        if(submission.selftext == ''): #skip any posts that contain no text
            continue
        elif(buySell == 0): #skip any posts that have a neutral sentiment
            continue
        elif any(word in stockCSV['Ticker'].values for word in wordsPost): #check if any of the words in the post are stock abbreviations
            postData.append({
                 #These are indentifying data
                'stock abbreviation': next(word for word in wordsPost if word in stockCSV['Ticker'].values),
                'stock name': stockCSV.loc[stockCSV['Ticker'] == next(word for word in wordsPost if word in stockCSV['Ticker'].values)]['Name'].values[0],
                'title': submission.title,
                'date': datetime.datetime.fromtimestamp(submission.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                'id': submission.id,
                'url': submission.url, 
                'post text': submission.selftext, #post text
                #All of the following are features that can be used to predict the stock price
                'upvote Ratio': submission.upvote_ratio, #upvote ratio
                'comment number': submission.num_comments, #number of comments
                'text length': len(submission.selftext),#length of the post text
                'flesch_kincaid_grade': textstat.flesch_kincaid_grade(submission.selftext), #Flesch Kincaid Grade
                'lexical_diversity': len(set(wordsPost)) / len(wordsPost), #lexical diversity
                'Sentiment (polarity):': blob.sentiment.polarity, #Sentiment
                'Sentiment (subjectivity)': blob.sentiment.subjectivity, #Subjectivity
                'Buy/Sell': buySell, #Buy or Sell
                'Buy/Sell Price': 0,
                'Price same time next day':0,
                'Delta': 0, 
                'Good/Bad Trade': "Good"
            })
        else:
            continue
        

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tyler\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [123]:
df = pd.DataFrame(postData)

df 

,stock abbreviation,stock name,title,date,id,url,post text,upvote Ratio,comment number,text length,flesch_kincaid_grade,lexical_diversity,Sentiment (polarity):,Sentiment (subjectivity),Buy/Sell,Buy/Sell Price,Price same time next day,Delta,Good/Bad Trade
0,$MSTR,Microstrategy Inc.,$MSTR / $MSTX bag holder,2024-11-26 17:20:56,1h0olph,https://i.redd.it/j1a5i3eqmb3e1.jpeg,Just joined the ranks of a bunch of $MSTR / $M...,0.77,23,71,2.5,0.928571,0.000000,0.000000,0,0,0,0,Good
1,$M,Macy's Inc,$M; Taking Advantage of Turkey Day,2024-11-26 14:50:14,1h0kyg2,https://i.redd.it/nfouf1kuva3e1.png,Every year Macy's sponsors the massive Thanksg...,0.87,3,501,7.3,0.853659,0.051245,0.340458,1,0,0,0,Good
2,$CBL,Cbl & Associated Properties,$CBL: The Shopping REIT that Screws Banks and ...,2024-11-25 23:05:07,1h031g7,https://www.reddit.com/r/wallstreetbets/commen...,***TL;DR at Bottom for my ADHD brethren.***\n\...,0.95,8,3109,6.7,0.637081,0.110194,0.516644,1,0,0,0,Good
3,$MSTR,Microstrategy Inc.,$MARA DD - The MaraStrategy 💊,2024-11-24 21:20:12,1gz8ajy,https://www.reddit.com/r/wallstreetbets/commen...,📢Preface\n\nDisclaimer: I use a cringe amount ...,0.85,176,9872,9.9,0.494743,0.097414,0.495243,1,0,0,0,Good


In [ ]:
#TO DO 
#1. Add Data from Yahoo Finance API to the dataframe
#2. Get the "buy/sell" value of that stock for the day
#3. See what happended a day later
#4. Get the Delta between the two values
#5. Use that to create a model that predicts the stock price
from datetime import timedelta, datetime, time
import pytz
import yfinance as yf


stockMarketHolidays2024 = ['2024-01-01', '2024-01-15', '2024-02-19', '2024-04-19', '2024-05-27', '2024-07-04', '2024-09-02', '2024-11-28', '2024-12-25']

x = 0


for index, row in df.iterrows():

    stock = yf.Ticker(row['stock abbreviation'][1:]) #remove the $ sign and gets the stock value
    
    timeBuy = row['date']
    
    timeBuy = datetime.strptime(timeBuy, '%Y-%m-%d %H:%M:%S') #converts the date to a datetime object


    if(timeBuy.time() > time(16, 0, 0)): #if the time is after 4pm, we need to get the next day's opening price     
        timeBuy = timeBuy +timeDelta(days=1) 
        timeBuy = datetime(timeBuy.year, timeBuy.month, timeBuy.day, 9, 31, 0) #sets the time to 9:30am
    elif(timeBuy.time() < time(9, 30,00)): #if the time is before 9:30am, we need to get the previous day's price
        timeBuy = timeBuy - timeDelta(days=1)
        timeBuy = datetime.datetime(timeBuy.year, timeBuy.month, timeBuy.day, 15, 59, 0)

    if(timeBuy.strftime('%Y-%m-%d') in stockMarketHolidays2024):
        timeBuy = timeBuy + timeDelta(days=1)


    timeBuy = timeBuy.replace(second=0, microsecond=0) #removes the seconds and microseconds

    timeBuy = str(str(timeBuy) +"-05:00") #converts the datetime object to a string
   

    hist = stock.history(period="5d", interval = '1m') #stores the prices for a whole day


    df.at[index, 'Buy/Sell Price'] = hist.loc[timeBuy]['Close'] #gets the price of the stock at the time of the post


    

C:\Users\Tyler\AppData\Local\Temp\ipykernel_49596\2431535655.py:41: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '381.9353942871094' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[index, 'Buy/Sell Price'] = hist.loc[timeBuy]['Close'] #gets the price of the stock at the time of the post


In [125]:
df

,stock abbreviation,stock name,title,date,id,url,post text,upvote Ratio,comment number,text length,flesch_kincaid_grade,lexical_diversity,Sentiment (polarity):,Sentiment (subjectivity),Buy/Sell,Buy/Sell Price,Price same time next day,Delta,Good/Bad Trade
0,$MSTR,Microstrategy Inc.,$MSTR / $MSTX bag holder,2024-11-26 17:20:56,1h0olph,https://i.redd.it/j1a5i3eqmb3e1.jpeg,Just joined the ranks of a bunch of $MSTR / $M...,0.77,23,71,2.5,0.928571,0.000000,0.000000,0,381.935394,0,0,Good
1,$M,Macy's Inc,$M; Taking Advantage of Turkey Day,2024-11-26 14:50:14,1h0kyg2,https://i.redd.it/nfouf1kuva3e1.png,Every year Macy's sponsors the massive Thanksg...,0.87,3,501,7.3,0.853659,0.051245,0.340458,1,15.995000,0,0,Good
2,$CBL,Cbl & Associated Properties,$CBL: The Shopping REIT that Screws Banks and ...,2024-11-25 23:05:07,1h031g7,https://www.reddit.com/r/wallstreetbets/commen...,***TL;DR at Bottom for my ADHD brethren.***\n\...,0.95,8,3109,6.7,0.637081,0.110194,0.516644,1,30.790001,0,0,Good
3,$MSTR,Microstrategy Inc.,$MARA DD - The MaraStrategy 💊,2024-11-24 21:20:12,1gz8ajy,https://www.reddit.com/r/wallstreetbets/commen...,📢Preface\n\nDisclaimer: I use a cringe amount ...,0.85,176,9872,9.9,0.494743,0.097414,0.495243,1,425.069885,0,0,Good


In [126]:
import os

file_path = 'reddit_data.csv'

# Check if the file exists
if not os.path.isfile(file_path):
    # If the file does not exist, write the DataFrame with the header
    df.to_csv(file_path, mode='w', header=True, index=False)
else:
    # If the file exists, append the DataFrame without the header
    df.to_csv(file_path, mode='a', header=False, index=False)

In [ ]:
newDF = pd.read_csv('reddit_data.csv')
for index, row in newDF.iterrows():
    stock = yf.Ticker(row['stock abbreviation'][1:]) #remove the $ sign and gets the stock value
    
    timeBuy = row['date']
    
    timeBuy = datetime.strptime(timeBuy, '%Y-%m-%d %H:%M:%S') #converts the date to a datetime object


    if(timeBuy.time() > time(16, 0, 0)): #if the time is after 4pm, we need to get the next day's opening price     
        timeBuy = timeBuy +timeDelta(days=1) 
        timeBuy = datetime(timeBuy.year, timeBuy.month, timeBuy.day, 9, 31, 0) #sets the time to 9:30am
    elif(timeBuy.time() < time(9, 30,00)): #if the time is before 9:30am, we need to get the previous day's price
        timeBuy = timeBuy - timeDelta(days=1)
        timeBuy = datetime.datetime(timeBuy.year, timeBuy.month, timeBuy.day, 15, 59, 0)


    timeBuy = timeBuy + timedelta(1)#add 1 day to the date

    if(timeBuy.strftime('%Y-%m-%d') in stockMarketHolidays2024):
        timeBuy = timeBuy + timeDelta(days=1)
        timeBuy = timeBuy.replace(second=0, microsecond=0)

    timeBuy = timeBuy.replace(second=0, microsecond=0) #removes the seconds and microseconds

    if timeBuy.weekday() == 5:  # if it's Saturday
        timeBuy = timeBuy + timedelta(days=2)
    elif timeBuy.weekday() == 6:  # if it's Sunday
        timeBuy = timeBuy + timedelta(days=1)

    if(timeBuy.date() == datetime(month=11, day= 27, year = 2024).date()):
        timeBuy = str(str(timeBuy) +"-01:00")
    elif(timeBuy.date() == datetime(month=11, day= 29, year = 2024).date()):
        print("RAN")
        timeBuy = str(str(timeBuy) +"-01:00")
    else:
         timeBuy = str(str(timeBuy) +"-05:00") 

    hist = stock.history(period="5d", interval = '1m')
    print(hist.head())

    idx = hist.index.get_indexer([timeBuy], method='nearest')
  
    # Fetch the closest timestamp and corresponding Close price
    closest_time = hist.index[idx[0]]
    close_price = hist.loc[closest_time, 'Close']

    # Update the DataFrame
    newDF.loc[index, 'Price same time next day'] = close_price
    newDF.loc[index, 'Delta'] = close_price - row['Buy/Sell Price']

    if newDF.loc[index, 'Buy/Sell'] == 1:
        if(newDF.loc[index, 'Delta'] < 0):
            newDF.loc[index, 'Good/Bad Trade'] = "Bad"
        else:
            newDF.loc[index, 'Good/Bad Trade'] = "Good"
    elif newDF.loc[index, 'Buy/Sell'] == -1:
        if(newDF.loc[index, 'Delta'] > 0):
            newDF.loc[index, 'Good/Bad Trade'] = "Bad"
        else:
            newDF.loc[index, 'Good/Bad Trade'] = "Good"
    else:
        newDF.loc[index, 'Good/Bad Trade'] = "Neutral"
        

    
    
    

RAN
                                 Open        High         Low       Close  \
Datetime                                                                    
2024-11-25 09:30:00-05:00  440.750000  440.750000  430.549988  432.609985   
2024-11-25 09:31:00-05:00  432.790009  433.264313  423.838715  425.069885   
2024-11-25 09:32:00-05:00  424.480011  430.649994  422.209991  427.224915   
2024-11-25 09:33:00-05:00  426.179993  428.399994  416.079987  417.440002   
2024-11-25 09:34:00-05:00  417.733704  418.000000  410.130005  417.920013   

                            Volume  Dividends  Stock Splits  
Datetime                                                     
2024-11-25 09:30:00-05:00  3586598        0.0           0.0  
2024-11-25 09:31:00-05:00   358491        0.0           0.0  
2024-11-25 09:32:00-05:00   324059        0.0           0.0  
2024-11-25 09:33:00-05:00   344648        0.0           0.0  
2024-11-25 09:34:00-05:00   286709        0.0           0.0  
                      

C:\Users\Tyler\AppData\Local\Temp\ipykernel_49596\390545375.py:49: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '404.0237121582031' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  newDF.loc[index, 'Price same time next day'] = close_price


                                Open       High        Low      Close  Volume  \
Datetime                                                                        
2024-11-25 09:30:00-05:00  29.620001  29.620001  29.620001  29.620001    2132   
2024-11-25 09:31:00-05:00  29.770000  29.770000  29.770000  29.770000     239   
2024-11-25 09:32:00-05:00  29.850000  29.850000  29.815001  29.815001     341   
2024-11-25 09:33:00-05:00  29.840000  29.959999  29.840000  29.959999     355   
2024-11-25 09:35:00-05:00  29.910000  29.959999  29.860001  29.910000     977   

                           Dividends  Stock Splits  
Datetime                                            
2024-11-25 09:30:00-05:00        0.4           0.0  
2024-11-25 09:31:00-05:00        0.0           0.0  
2024-11-25 09:32:00-05:00        0.0           0.0  
2024-11-25 09:33:00-05:00        0.0           0.0  
2024-11-25 09:35:00-05:00        0.0           0.0  
                                 Open        High         Low

In [ ]:
newDF.add

,stock abbreviation,stock name,title,date,id,url,post text,upvote Ratio,comment number,text length,flesch_kincaid_grade,lexical_diversity,Sentiment (polarity):,Sentiment (subjectivity),Buy/Sell,Buy/Sell Price,Price same time next day,Delta,Good/Bad Trade
0,$MSTR,Microstrategy Inc.,$MSTR / $MSTX bag holder,2024-11-26 17:20:56,1h0olph,https://i.redd.it/j1a5i3eqmb3e1.jpeg,Just joined the ranks of a bunch of $MSTR / $M...,0.78,23,71,2.5,0.928571,0.000000,0.000000,0,381.935394,404.023712,0,Good
1,$M,Macy's Inc,$M; Taking Advantage of Turkey Day,2024-11-26 14:50:14,1h0kyg2,https://i.redd.it/nfouf1kuva3e1.png,Every year Macy's sponsors the massive Thanksg...,0.84,3,501,7.3,0.853659,0.051245,0.340458,1,15.995000,16.025000,0,Good
2,$CBL,Cbl & Associated Properties,$CBL: The Shopping REIT that Screws Banks and ...,2024-11-25 23:05:07,1h031g7,https://www.reddit.com/r/wallstreetbets/commen...,***TL;DR at Bottom for my ADHD brethren.***\n\...,0.94,8,3109,6.7,0.637081,0.110194,0.516644,1,30.790001,31.180000,0,Good
3,$MSTR,Microstrategy Inc.,$MARA DD - The MaraStrategy 💊,2024-11-24 21:20:12,1gz8ajy,https://www.reddit.com/r/wallstreetbets/commen...,📢Preface\n\nDisclaimer: I use a cringe amount ...,0.85,176,9872,9.9,0.494743,0.097414,0.495243,1,425.069885,380.611908,0,Good
4,$MSTR,Microstrategy Inc.,$MSTR / $MSTX bag holder,2024-11-26 17:20:56,1h0olph,https://i.redd.it/j1a5i3eqmb3e1.jpeg,Just joined the ranks of a bunch of $MSTR / $M...,0.77,23,71,2.5,0.928571,0.000000,0.000000,0,381.935394,404.023712,0,Good
5,$M,Macy's Inc,$M; Taking Advantage of Turkey Day,2024-11-26 14:50:14,1h0kyg2,https://i.redd.it/nfouf1kuva3e1.png,Every year Macy's sponsors the massive Thanksg...,0.87,3,501,7.3,0.853659,0.051245,0.340458,1,15.995000,16.025000,0,Good
6,$CBL,Cbl & Associated Properties,$CBL: The Shopping REIT that Screws Banks and ...,2024-11-25 23:05:07,1h031g7,https://www.reddit.com/r/wallstreetbets/commen...,***TL;DR at Bottom for my ADHD brethren.***\n\...,0.95,8,3109,6.7,0.637081,0.110194,0.516644,1,30.790001,31.180000,0,Good
7,$MSTR,Microstrategy Inc.,$MARA DD - The MaraStrategy 💊,2024-11-24 21:20:12,1gz8ajy,https://www.reddit.com/r/wallstreetbets/commen...,📢Preface\n\nDisclaimer: I use a cringe amount ...,0.85,176,9872,9.9,0.494743,0.097414,0.495243,1,425.069885,380.611908,0,Good
